В обработке сигналов может пригодиться библиотека `scipy`. По ссылке можно ознакомиться с документацией и со списком модулей, входящих в библиотеку:  
https://docs.scipy.org/doc/scipy/tutorial/index.html#user-guide

Модуль `scipy.fft` содержит функции, относящиеся к преобразованию Фурье. Нарисуем спектр Фурье для суммы двух синусоид (пример из документации `scipy`). Нам понадобится собственно функция `fft`, которая вычисляет быстрое преобразование Фурье для дискретного сигнала, и функция `fftfreq`, которая возвращает значения частоты (в Герцах), которые мы будем откладывать по оси X на спектрограмме.

In [11]:
from scipy.fft import fft, fftfreq
import numpy as np

# Количество отсчётов в сигнале
N = 600

# Период дискретизации (в секундах)
T = 1.0 / 800.0

# Заведём массив временных точек (N отсчётов с промежутком T)
x = np.linspace(0.0, N*T, N, endpoint=False)
# Построим наш сигнал как сумму двух синусоид с частотами 50 и 80 Гц
y = np.sin(50.0 * 2.0*np.pi*x) + 0.5*np.sin(80.0 * 2.0*np.pi*x)

Посмотрим отдельно на полученные массивы:

In [ ]:
# это значения времени каждого отсчёта сигнала
print(x[:20])

In [ ]:
# это значения амплитуды каждого отсчёта сигнала
print(y[:20])

Нарисуем наш сигнал на графике:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(x, y)
plt.xlabel("Time, s")
plt.ylabel("Amplitude")
plt.show()

Займёмся вычислением спектра:

In [ ]:
# Вычислим преобразование Фурье
yf = fft(y)
# Получим список частот (в Гц)
xf = fftfreq(N, T)[:N//2]

Посмотрим, что получилось:

In [ ]:
# Это то, что мы будем откладывать по оси X на спектре (значения частот):
print(xf[:20])

In [ ]:
# А это значения спектра в каждой частоте:
print(yf[:20])

Вместо `fft` и `fftfreq` можно воспользоваться `rfft` и `rfftfreq`, тогда не надо будет брать половинки от массивов:

In [12]:
from scipy.fft import rfft, rfftfreq
# Вычислим преобразование Фурье
yf_r = rfft(y)
# Получим список частот (в Гц)
xf_r = rfftfreq(N, T)

Обратите внимание, что значения спектра &ndash; это комплексные числа!

Нарисуем спектр на графике:

In [ ]:
# Возьмём модуль от нашего спектра, чтобы получить вещественные числа
plt.plot(xf_r, 2.0/N * np.abs(yf_r))
plt.xlabel("Frequency, Hz")
plt.ylabel("Amplitude")
plt.grid()
plt.show()

Чтобы краевые эффекты не вносили искажения в спектр, полезно умножить значения в сигнале на какую-нибудь оконную функцию. Для этого в модуле `scipy.signal` есть пространство имён `windows` и функция `get_window`. Со списком окон можно ознакомиться в документации:  
https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.get_window.html

Например, получим окно Ханна под наш сигнал:

In [ ]:
from scipy import signal
hann_window = signal.get_window("hann", N)
plt.plot(x, hann_window)
plt.xlabel("Time, s")
plt.ylabel("Amplitude")
plt.show()

Мы можем просто умножить сигнал на окно:

In [ ]:
y_windowed = y * hann_window
plt.plot(x, y_windowed)
plt.xlabel("Time, s")
plt.ylabel("Amplitude")
plt.show()

Заново вычислим преобразование Фурье:

In [ ]:
yf_windowed = rfft(y_windowed)
plt.plot(xf_r, 2.0/N * np.abs(yf_windowed))
plt.xlabel("Frequency, Hz")
plt.ylabel("Amplitude")
plt.grid()
plt.show()

**Задание для выполнения в классе**: прочитайте файл cta0001.wav с помощью `scipy.io.wavfile` и нарисуйте на графике спектр Фурье фрагмента сигнала, который начинается с 200 мс и длится 30 мс. Используйте любую оконную функцию.

In [ ]:
!wget https://pkholyavin.github.io/mastersprogramming/cta0001.wav

Обработка сигнала окнами (фреймами)

При обработке сигнала часто используют т.н. оконный метод: некоторый признак вычисляют на небольшом фрагменте сигнала, на котором мы считаем его условно постоянным. Мы делим сигнал длиной N<sub>total</sub> на K таких фрагментов, причём каждый фрагмент (фрейм) накладывается на своих соседей. Основные параметры:

N - длина окна (в отсчётах)  
S - шаг (в отсчётах)  

**Вопрос:** как вычислить количество окон, на которые можно разделить сигнал?

<details>
<summary>Ответ</summary>

$$K = 1 + \left \lfloor{\frac{N_{total} - N}{S}}\right \rfloor$$

</details>


**Вопрос 2:** а что делать, если сигнал не делится на целое количество окон?

<details>
<summary>Ответ</summary>

1. Отрезать лишнее
2. Добавить нулевых отсчётов до целого

</details>

**Вопрос 3:** если мы задали длину окна в миллисекундах, как понять, сколько в нём будет отсчётов?

<details>
<summary>Ответ</summary>

$$N_{samples} = round(\frac{N_{ms} \cdot {F_{s}}}{1000})$$

</details>


**Задание для выполнения в классе**: обработайте сигнал сta0001.wav окнами по 30 мс с шагом 10 мс, на каждом окне вычислите максимум амплитуды. Постройте график.

**Домашнее задание:**

Напишите программу, которая считывает файл .wav и строит динамическую спектрограмму. Для этого:
1. Нормализуйте сигнал (поделите на максимум для int16)
2. Задайте двумерный массив (матрицу), в которой будет столько же столбцов, сколько окон в сигнале, и столько же строк, сколько отсчётов в вашем спектре (т.е. N // 2)
3. Обработайте сигнал окнами (задайте длину окна и шаг)
4. Постройте power spectrum (спектр Фурье, возведённый в квадрат) каждого окна
5. Заполните массив полученными спектрами
6. Используйте функцию imshow для визуализации

In [ ]:
import random
x = 100
y = 200
heatmap = np.asarray([[random.randint(0, 10) * i * j for i in range(x)] for j in range(y)])
plt.imshow(heatmap)